<a href="https://colab.research.google.com/github/Prem-Healthinformatics/Drug_intersect/blob/main/Drug_intersect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB  # Using Naive Bayes for less memory consumption
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import joblib

In [2]:
medicine_details = pd.read_csv('/content/Medicine_Details.csv')  # Specify needed columns
medical_intelligence = pd.read_csv('/content/Medical_Intelligence_Dataset.csv')
ai_chatbot = pd.read_csv('/content/ai-medical-chatbot.csv', nrows=10000)  # Consider using a smaller number

In [3]:
# Step 3: Data Preparation
# Clean the datasets
medicine_details.fillna('', inplace=True)
medical_intelligence.fillna('', inplace=True)
ai_chatbot.fillna('', inplace=True)

# Prepare feature and target variables
X = medical_intelligence['input']  # Features
y = medical_intelligence['output']  # Target

In [4]:
# Step 4: Train the Model
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a pipeline with TF-IDF and Naive Bayes
model = make_pipeline(TfidfVectorizer(max_features=5000), MultinomialNB())  # Reduce max features

# Train the model
model.fit(X_train, y_train)

# Save the trained model
joblib.dump(model, 'medicine_recommendation_model.pkl')
print("Model saved successfully!")

# Evaluate model accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")

Model saved successfully!
Model accuracy: 0.82%


In [5]:
# Step 5: Load the Model
loaded_model = joblib.load('medicine_recommendation_model.pkl')
print("Model loaded successfully!")

Model loaded successfully!


In [6]:
# Step 6: Create Functions for Interaction
def predict_medicine(symptoms):
    """Predict medicine based on symptoms."""
    prediction = loaded_model.predict([symptoms])
    return prediction[0]

def get_medicine_info(medicine):
    """Get uses and side effects of a given medicine."""
    medicine_info = medicine_details.loc[medicine_details['Medicine Name'] == medicine]
    if not medicine_info.empty:
        uses = medicine_info['Uses'].values[0]
        side_effects = medicine_info['Side_effects'].values[0]
        return uses, side_effects
    return "No information found.", "No side effects found."

def recommend_alternatives(prescribed_medicine):
    """Recommend alternative medicines based on similar uses and fewer side effects."""
    # Get the uses and side effects of the prescribed medicine
    prescribed_info = medicine_details[medicine_details['Medicine Name'] == prescribed_medicine]
    if prescribed_info.empty:
        return pd.DataFrame()  # Return empty dataframe if no info found

    prescribed_uses = prescribed_info['Uses'].values[0].split(',')
    prescribed_side_effects = prescribed_info['Side_effects'].values[0].split(',')

    # Filter medicines with similar uses
    def has_similar_use(row):
        row_uses = row['Uses'].split(',')
        return any(use in row_uses for use in prescribed_uses)

    alternatives = medicine_details[medicine_details['Medicine Name'] != prescribed_medicine]
    alternatives = alternatives[alternatives.apply(has_similar_use, axis=1)]

    # Rank alternatives by fewer side effects
    alternatives['Side_effect_count'] = alternatives['Side_effects'].apply(lambda x: len(x.split(',')))
    prescribed_side_effect_count = len(prescribed_side_effects)

    # Select alternatives with fewer or equal side effects
    alternatives = alternatives[alternatives['Side_effect_count'] <= prescribed_side_effect_count]

    # Sort by side effect count and limit the number of alternatives
    alternatives = alternatives.sort_values(by='Side_effect_count').head(5)

    # Drop the side effect count column before returning
    return alternatives[['Medicine Name', 'Uses', 'Side_effects']]


def get_side_effects(medicine):
    """Get side effects of a given medicine."""
    side_effects = medicine_details.loc[medicine_details['Medicine Name'] == medicine, 'Side_effects']
    return side_effects.values[0] if not side_effects.empty else "No side effects found."


In [7]:
# Step 7: Parallel Processing for User Input
def handle_user_requests(user_input):
    """Handle user requests using parallel processing."""
    if user_input in medicine_details['Medicine Name'].values:
        # User provided a medicine name
        uses, side_effects = get_medicine_info(user_input)
        alternatives = recommend_alternatives(user_input)
        return f"Uses: {uses}\nSide Effects: {side_effects}", alternatives
    else:
        # User provided symptoms
        recommended_medicine = predict_medicine(user_input)
        return f"Recommended Medicine: {recommended_medicine}", recommend_alternatives(recommended_medicine)


In [8]:
# Step 8: Collect User Inputs
user_input = input("Enter a medicine name: ")

# Step 9: Process Requests and Display Results
response, alternatives = handle_user_requests(user_input)

# Check if the input is a medicine name or symptoms
if user_input in medicine_details['Medicine Name'].values:
    print(f"Information for {user_input}:\n{response}")
else:
    print(response)  # For symptoms, this will show the recommended medicine

# Print recommended alternatives with clear headers
print("Recommended Alternatives:")
if not alternatives.empty:
    for index, row in alternatives.iterrows():
        print(f"\n- Medicine Name: {row['Medicine Name']}")
        print(f"  Uses: {row['Uses']}")
        print(f"  Side Effects: {row['Side_effects']}")
else:
    print("No alternatives found.")


Enter a medicine name: Augmentin 625 Duo Tablet
Information for Augmentin 625 Duo Tablet:
Uses: Treatment of Bacterial infections
Side Effects: Vomiting Nausea Diarrhea Mucocutaneous candidiasis
Recommended Alternatives:

- Medicine Name: Zenflox Suspension
  Uses: Treatment of Bacterial infections
  Side Effects: Nausea Headache Dizziness Itching Insomnia difficulty in sleeping Rash Genital itching Vaginal inflammation Phlebitis Injection site reactions pain swelling redness

- Medicine Name: Zifi CV 100 Dry Syrup
  Uses: Treatment of Bacterial infections
  Side Effects: Nausea Diarrhea Stomach pain Indigestion Flatulence

- Medicine Name: Zithrox 200 Readyuse Suspension
  Uses: Treatment of Bacterial infections
  Side Effects: Nausea Abdominal pain Diarrhea

- Medicine Name: Ziprax 200 DT Tablet
  Uses: Treatment of Bacterial infections
  Side Effects: Nausea Stomach pain Indigestion Diarrhea

- Medicine Name: Zostum O 100mg Dry Syrup
  Uses: Treatment of Bacterial infections
  Side 

In [9]:
!git config --global user.name "Prem_Healthinformatics"
!git config --global user.email "Kumarprem192000@gmail.com"